In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# <div style="color:white;display:fill;border-radius:5px;background-color:#75B7BF;letter-spacing:0.1px;overflow:hidden"><p style="padding:20px;color:white;overflow:hidden;margin:0;font-size:100%;text-align:center">Introduction</p></div>

Bacterial genetics is the study of the structure and distribution of hereditary information in bacteria. Although many species are often considered harmful, some can actually be helpful. In fact, we would not exist without them. The study of bacteria has been used across many industries to provide a variety of benefits from creating new antibiotics and medications to developing genetic enzymes that can break down organic compounds, like plastic. In this analysis, I will examine the genetic information of 10 different species of bacteria using Principal Components Analysis and build a predictive model that can classify bacterial species with 96% accuracy. 

<img src="https://media.istockphoto.com/vectors/icon-vector-template-illustration-design-vector-id1194702268?k=20&m=1194702268&s=612x612&w=0&h=BTdNZSjkEFNxqVAB5bF0Cdfkil7oiL70PISp7235FbM=" width=600>

# <div style="color:white;display:fill;border-radius:5px;background-color:#75B7BF;letter-spacing:0.1px;overflow:hidden"><p style="padding:20px;color:white;overflow:hidden;margin:0;font-size:100%;text-align:center">Data Overview</p></div>

The [data](https://www.kaggle.com/c/tabular-playground-series-feb-2022/data) used in this study is based on compressed measurements of DNA snippets. Snippets of length 10 were analyzed using Raman spectroscopy that calculates the histogram of bases in the snippet. Using this technique, the DNA segment $ATATGGCCTT$ becomes translated to $A_2T_4G_2C_2$.  

Each row of data contains the spectrum of histograms generated by repeated measurements of the sample. The output of all 286 histogram possibilities ($A_0T_0G_0C_{10}$ to $A_{10}T_0G_0C_0$) then has a bias spectrum (of totally random $ATGC$) subtracted from the results.  

The training set consists of 200,000 bacteria across 10 different species including Bacteroides fragilis, Campylobacter jejuni, Enterococcus hirae, Escherichia coli, Escherichia fergusonii, Klebsiella pneumoniae, Salmonella enterica, Staphylococcus aureus, Streptococcus pneumoniae, Streptococcus pyogenes. The data set also contains several duplicated rows. These will be removed before beginning the analysis.

In [ ]:
# Load libraries
import os, warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as mpl
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score, auc, classification_report

# Load data
train = pd.read_csv('../input/tabular-playground-series-feb-2022/train.csv', index_col=0)
test = pd.read_csv('../input/tabular-playground-series-feb-2022/test.csv', index_col=0)

print('Train Shape: {}\nMissing Data: {}\nDuplicates: {}\n'\
      .format(train.shape, train.isna().sum().sum(), train.duplicated().sum()))
print('Test Shape: {}\nMissing Data: {}\nDuplicates: {}\n'\
      .format(test.shape, test.isna().sum().sum(), test.duplicated().sum()))

### <b><span style='color:#53A0AA'>Check Duplication and Dropping it</span> </b>

In [ ]:
train_df=train.drop_duplicates() 
print('Dropping Duplicates\nNew Train Shape: {}'.format(train_df.shape))

### <b><span style='color:#53A0AA'>Check Null Available</span> </b>

In [ ]:
missing_val = train_df.isnull().sum()
missing_val

## <b><span style='color:#53A0AA'>Summary Statistics Grouped by Species</span> </b>

In [ ]:
train_df.groupby('target').describe()

In [ ]:
target_length = train_df.target.value_counts().shape[0]

# <div style="color:white;display:fill;border-radius:5px;background-color:#75B7BF;letter-spacing:0.1px;overflow:hidden"><p style="padding:20px;color:white;overflow:hidden;margin:0;font-size:100%;text-align:center">Exploratory Data Analysis</p></div>

In [ ]:
target_df = pd.DataFrame(train_df.target.value_counts()).reset_index()
target_df.columns = ['target', 'count']
fig = px.bar(data_frame =target_df, 
             x = 'target',
             y = 'count' , 
             color = "count",
             color_continuous_scale="Emrld") 
fig.update_layout(template = "plotly_white")
for idx,target in enumerate(target_df["target"]):
    print("\033[94mPercentage of " + str(target) + " category  : {:.2f} %".format(target_df["count"][idx] *100 / train.shape[0]))
fig.show()

In [ ]:
target_length = train_df.target.value_counts().shape[0]
init_notebook_mode(connected=True)
pal = sns.color_palette("mako_r", 12).as_hex()[:target_length]
bact=train_df.target.value_counts(normalize=True).reset_index()
bact.target=bact.target.mul(100).sort_values(ascending=False)
bact['index']=bact['index'].str.replace('_', ' ') 

temp = dict(layout=go.Layout(font=dict(family="Franklin Gothic", size=12)))
fig = px.bar(bact, x='index', y='target', text='target', color='index', 
             color_discrete_sequence=pal, opacity=0.8)
fig.update_traces(texttemplate='%{text:,.2f}%', textposition='outside',
                  marker_line=dict(width=1, color='#28221D'))
fig.update_yaxes(visible=False, showticklabels=False)
fig.update_layout(template=temp, title_text='Distribution of Bacteria Species', 
                  xaxis=dict(title='', tickangle=25, showline=True), 
                  height=450, width=700, showlegend=False)
fig.show()

## <b><span style='color:#53A0AA'>Line Plot</span> </b>

In [ ]:
def plot_line(name):
    df_name = train_df.iloc[np.where(train_df.target==name)]
    for i in range(target_length):
        df_name.iloc[i][:-2].plot(kind='line', figsize=(20,6))
    plt.show()
plot_line('Escherichia_fergusonii')
    

## <b><span style='color:#53A0AA'>Correlations</span> </b>

In [ ]:
cor = train_df.corr()
cor.style.background_gradient(cmap='viridis')

## <b><span style='color:#53A0AA'>The Most Correlated Genes in Each Species</span></b>

In [ ]:
for i in train_df.target.unique():
    cor_df = train_df[train_df.target==i]
    cor = cor_df.corr()  
    c = cor.abs().unstack().drop_duplicates().reset_index()
    c = c.rename(columns={'level_0': 'Gene 1', 'level_1': 'Gene 2', 0: 'Correlation'})
    c = c.query('Correlation < 1').sort_values(by = 'Correlation', ascending = False).reset_index(drop=True)
    display(c.iloc[:1,:].style.background_gradient(cmap='flare').set_caption('Most correlated genes in {}'.format(i.replace('_', ' '))))

## <b><span style='color:#53A0AA'>The Most Correlated Gen Pairs</span> </b>

In [ ]:
c = cor.abs().unstack().drop_duplicates().reset_index()
c = c.rename(columns={'level_0': 'Gene 1', 'level_1': 'Gene 2', 0: 'Correlation'})
c = c.query('.75 <= Correlation < 1').sort_values(by = 'Correlation', ascending = False).reset_index(drop=True)
c.style.background_gradient(cmap='flare_r')

# <div style="color:white;display:fill;border-radius:5px;background-color:#75B7BF;letter-spacing:0.1px;overflow:hidden"><p style="padding:20px;color:white;overflow:hidden;margin:0;font-size:100%;text-align:center">Reference</p></div>

https://www.kaggle.com/phanttan/genetic-analysis-of-bacteria-with-pca/

https://www.kaggle.com/teckmengwong/tpsfeb22-soft-voting-baseline